In [ ]:
import numpy as np
from numpy import pi
from oracles import simon_oracle
from qiskit import QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

Algorytm Simona to pierwszy algorytm kwantowy, który pozwolił na uzyskanie eksponencjalnego przyspieszenia rozwiązania pewnego problemu matematycznego, a mianowicie problemu Simona. Nie istnieje klasyczne rozwiązanie tego zagadnienia o złożoności obliczeniowej mniejszej od wykładniczej. Natomiast złożoność algorytmu kwantowego jest liniowa.  

### Problem Simona
Dana jest czarna skrzynka (*black box*) implementująca funkcję $f:\{0,1\}^n \rightarrow \{0,1\}^n$. O funkcji $f$ wiemy, że jest 1-na-1 lub 2-na-1.

**Zadanie**. Podać różne przykłady funkcji 1-na-1 oraz 2-na-1.
- czy $g: \mathbb{R} \rightarrow\mathbb{R}, g(x) = x$ jest 1-na-1 lub 2-na-1?
- czy $h: \mathbb{R} \rightarrow\mathbb{R}, h(x) = x^2$ jest 1-na-1 lub 2-na-1 ?

Funkcja $f$ w problemie Simona jest określona przez ukryty ciąg $s \in \{0,1\}^n $ taki że:  
$f(x) = f(y) \iff x \oplus y=s$  

**Zadanie.** Podać przykład funkcji $f:\{0,1\}^3 \rightarrow \mathbb{Z}$, która spełnia warunek $f(x + 101) = f(x)$.  
**Zadanie.** Podać przykład funkcji $f:\{0,1\}^3 \rightarrow \{0,1\}^3$, która spełnia warunek $f(x + 101) = f(x)$.   
  
**Problem Simona.** Jak efektywnie określić czy funkcja $f$ jest 1-na-1,  czy jest 2-na-1? Jak efektywnie znaleźć ukryty ciąg $s$?   


**Pytanie.** Jak klasycznie podejść do rozwiązania tego problemu? Ile zapytań do czarnej skrzynki (ile wywołań funkcji $f$) potrzeba w najlepszym / najgorszym przypadku? Jaka jest złożoność obliczeniowa?  


### Problem Simona - implementacja kwantowej wyroczni

**Zadanie:** Zaimplementować funkcję `simon_oracle`, która będzie zwracać jako wynik działania obwód kwantowy, wykonujący operację: 
$U_f: \vert x \rangle \vert 0...0 \rangle \rightarrow \vert x \rangle \vert f(x) \rangle$,
gdzie $f(x)$ jest funkcją opisaną powyżej, tj. spełnia warunek $f(x) = f(y) \iff x \oplus y=s$.

In [ ]:
# Sprawdzenie działania simons_oracle
s = '101'
n = len(s)
circuit = QuantumCircuit(2*n)

# Ustalenie argumentów wejściowych
circuit.x(0)
circuit.x(2)

# Sprawdzenie stanu wejściowego
statevector_1 = Statevector.from_instruction(circuit)
print(statevector_1.measure()[0][::-1])

# Dodanie funkcji (Simon's oracle)
# circuit.barrier()
circuit = circuit.compose(simon_oracle(s))

# Sprawdzenie stanu wyjściowego
statevector_2 = Statevector.from_instruction(circuit)
print(statevector_2.measure()[0][::-1])
# circuit.draw(output="mpl")

### Algorytm Simona - kwantowe rozwiązanie problemu Simona



In [ ]:
simon_circuit = QuantumCircuit(2*n, n)

# Przygotowanie obwodu

# Aplikacja funkcji Simona

# Transformacje "za wyrocznią"

# Wykonanie pomiaru na odpowiednich kubitach

simon_circuit.draw(output="mpl")

### Wykonanie pomiarów

In [ ]:
simulator = Aer.get_backend('aer_simulator')
result = simulator.run(simon_circuit, shots=2048).result()
counts = result.get_counts()
plot_histogram(counts)

### Analiza klasyczna - odszyfrowanie wyniku

Aby znaleźć ukryty ciąg $s$, trzeba skorzystać z faktu, że dla każdego z mierzonych wyników  $s\cdot z  = 0 \pmod{2}$.
To znaczy, że dostajemy układ równań:  
$s \cdot 101 = 0$  
$s \cdot 111 = 0$  
$s \cdot 000 = 0$  
$s \cdot 010 = 0$  
z którego musimy wyznaczyć $s$. Można go rozwiązać na przykład metodą eliminacji Gaussa.  
   
Wspomniany wyżej fakt możemy też wykorzystać do weryfikacji czy nasza implementacja algorytmu działa poprawnie. Z racji, że znamy już ukryty ciąg $s$, możemy sprawdzić czy rzeczywiście dla każdego z mierzonych wyników zachodzi $s\cdot z  = 0 \pmod{2}$.  

In [ ]:
# Sprawdzenie czy mierzymy poprawne stany (wszystkie wyniki powinny być równe zero).

def dot_product(x, y):
    result = 0
    for i in range(len(x)):
        result += int(x[i]) * int(y[i])
    return (result % 2)

for z in counts:
    print(f'({s})*({z}) = {dot_product(s,z)} (mod 2)')